In [21]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit.library import StatePreparation
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_state_city


In [38]:
def int_to_bits(a, num_bits=None):
    """Decompose integer a into an array of bits (MSB first).
    If num_bits is specified, the result is zero-padded to that length."""
    bits = [(a >> i) & 1 for i in range(num_bits or a.bit_length())]
    return bits  # LSB first

def remove_leading_0s(bits):
    first_one = next((i for i, b in enumerate(bits) if b), len(bits))
    return bits[first_one:]

def add_classically_controlled_X(circuit, qs, bit):
    if bit:
        for q in qs:
            circuit.x(q)

def addMAJ(circuit, q1, q2, q3, bit):
    if bit:
        circuit.x(q1)
        circuit.x(q2)
    
    circuit.ccx(q1,q2,q3)

    if bit:
        circuit.x(q1)
        circuit.x(q2)
        circuit.x(q3)

def addConstant(a, qubit_count):
    if a == 0:
        return QuantumCircuit(qubit_count)
    
    a_bits = int_to_bits(a, num_bits=qubit_count)
    a_bits = remove_leading_0s(a_bits)
    bit_count = len(a_bits)
    offset = qubit_count - bit_count  # number of new LSB qubits to prepend

    adderCircuit = addConstantClean(a_bits, bit_count)

    # Build expanded circuit: full-width data register + same ancilla register
    new_data = QuantumRegister(qubit_count, 'data')
    anc      = AncillaRegister(bit_count - 2, 'anc')
    result   = QuantumCircuit(new_data, anc, name=f"+{int(''.join(map(str, a_bits)), 2)}")

    # Map adderCircuit's data qubits onto the most-significant slots of new_data
    # (leaving the 'offset' least-significant qubits idle), ancillas map 1-to-1
    qubit_map = [new_data[offset + i] for i in range(bit_count)] + list(anc)
    result.compose(adderCircuit, qubits=qubit_map, inplace=True)

    return result

def addConstantClean(a_bits, qubit_count):
    data = QuantumRegister(qubit_count, 'data')
    anc  = AncillaRegister(qubit_count - 2, 'anc')
    qc   = QuantumCircuit(data, anc, name=f"+{int(''.join(map(str, a_bits)), 2)}")

    add_classically_controlled_X(qc, [data[0], data[1]], a_bits[1])
    qc.ccx(data[0], data[1], anc[0])

    add_classically_controlled_X(qc, [anc[0]], a_bits[1]^a_bits[2])
    add_classically_controlled_X(qc, [data[2]], a_bits[2])
    qc.ccx(anc[0], data[2], anc[1])

    add_classically_controlled_X(qc, [anc[1]], a_bits[2]^a_bits[3])
    add_classically_controlled_X(qc, [data[3]], a_bits[3])
    qc.ccx(anc[1], data[3], anc[2])

    add_classically_controlled_X(qc, [anc[2]], a_bits[3]^a_bits[4])
    add_classically_controlled_X(qc, [data[4]], a_bits[4])
    qc.ccx(anc[2], data[4], anc[3])

    add_classically_controlled_X(qc, [anc[3]], a_bits[4]^a_bits[5])
    add_classically_controlled_X(qc, [data[5]], a_bits[5])
    qc.ccx(anc[3], data[5], anc[4])

    add_classically_controlled_X(qc, [anc[4]], a_bits[5])
    qc.cx(anc[4], data[6])
    add_classically_controlled_X(qc, [anc[4]], a_bits[5])

    qc.ccx(anc[3], data[5], anc[4])
    add_classically_controlled_X(qc, [data[5], anc[3]], a_bits[5])

    qc.cx(anc[3], data[5])
    add_classically_controlled_X(qc, [anc[3]], a_bits[4])

    qc.ccx(anc[2], data[4], anc[3])
    add_classically_controlled_X(qc, [anc[2], data[4]], a_bits[4])

    qc.cx(anc[2], data[4])
    add_classically_controlled_X(qc, [anc[2]], a_bits[3])

    qc.ccx(anc[1], data[3], anc[2])
    add_classically_controlled_X(qc, [anc[1], data[3]], a_bits[3])

    qc.cx(anc[1], data[3])
    add_classically_controlled_X(qc, [anc[1]], a_bits[2])

    qc.ccx(anc[0], data[2], anc[1])
    add_classically_controlled_X(qc, [anc[0], data[2]], a_bits[2])

    qc.cx(anc[0], data[2])
    add_classically_controlled_X(qc, [anc[0]], data[1])

    qc.ccx(data[0], data[1], anc[0])
    add_classically_controlled_X(qc, [data[0], data[1]], data[1])

    qc.cx(data[0], data[1])

    qc.x(data[0])
    add_classically_controlled_X(qc, [data[1]], a_bits[1])
    add_classically_controlled_X(qc, [data[2]], a_bits[2])
    add_classically_controlled_X(qc, [data[3]], a_bits[3])
    add_classically_controlled_X(qc, [data[4]], a_bits[4])
    add_classically_controlled_X(qc, [data[5]], a_bits[5])
    add_classically_controlled_X(qc, [data[6]], a_bits[6])

    return qc


In [40]:
addConstant(150, 8).draw()

»
data_0: ──────────────────────────────────────────────────────────────────────»
        ┌───┐                                                                 »
data_1: ┤ X ├──■──────────────────────────────────────────────────────────────»
        ├───┤  │                                                              »
data_2: ┤ X ├──■──────────────────────────────────────────────────────────────»
        └───┘  │                                                              »
data_3: ───────┼─────────■────────────────────────────────────────────────────»
        ┌───┐  │         │                                                    »
data_4: ┤ X ├──┼─────────┼─────────■──────────────────────────────────────────»
        └───┘  │         │         │                                          »
data_5: ───────┼─────────┼─────────┼─────────■─────────────────────────────■──»
               │         │         │         │                      ┌───┐  │  »
data_6: ───────┼─────────┼─────────┼─────────┼────■─────────■───────┤ X ├──┼──»
               │         │         │         │    │  ┌───┐  │  ┌───┐└─┬─┘  │  »
data_7: ───────┼─────────┼─────────┼─────────┼────┼──┤ X ├──┼──┤ X ├──┼────┼──»
             ┌─┴─┐┌───┐  │         │         │    │  └─┬─┘  │  └───┘  │    │  »
 anc_0: ─────┤ X ├┤ X ├──■─────────┼─────────┼────┼────┼────┼─────────┼────┼──»
             └───┘└───┘┌─┴─┐┌───┐  │         │    │    │    │         │    │  »
 anc_1: ───────────────┤ X ├┤ X ├──■─────────┼────┼────┼────┼─────────┼────┼──»
                       └───┘└───┘┌─┴─┐┌───┐  │    │    │    │         │    │  »
 anc_2: ─────────────────────────┤ X ├┤ X ├──■────┼────┼────┼─────────┼────■──»
                                 └───┘└───┘┌─┴─┐  │    │    │         │  ┌─┴─┐»
 anc_3: ───────────────────────────────────┤ X ├──■────┼────■─────────■──┤ X ├»
                                           └───┘┌─┴─┐  │  ┌─┴─┐          └───┘»
 anc_4: ────────────────────────────────────────┤ X ├──■──┤ X ├───────────────»
                                                └───┘     └───┘               »
«                                                                         
«data_0: ─────────────────────────────────────────────────────────────────
«                                                          ┌───┐     ┌───┐
«data_1: ───────────────────────────────────────────────■──┤ X ├──■──┤ X ├
«                                                       │  ├───┤┌─┴─┐├───┤
«data_2: ───────────────────────────────────────────────■──┤ X ├┤ X ├┤ X ├
«                                           ┌───┐       │  └───┘└───┘└───┘
«data_3: ───────────────────────────■───────┤ X ├───────┼─────────────────
«                       ┌───┐┌───┐  │  ┌───┐└─┬─┘       │                 
«data_4: ────────────■──┤ X ├┤ X ├──┼──┤ X ├──┼─────────┼─────────────────
«        ┌───┐       │  └───┘└─┬─┘  │  └───┘  │         │                 
«data_5: ┤ X ├───────┼─────────┼────┼─────────┼─────────┼─────────────────
«        └─┬─┘       │         │    │         │         │                 
«data_6: ──┼─────────┼─────────┼────┼─────────┼─────────┼─────────────────
«          │         │         │    │         │         │                 
«data_7: ──┼─────────┼─────────┼────┼─────────┼─────────┼─────────────────
«          │         │         │    │         │  ┌───┐┌─┴─┐               
« anc_0: ──┼─────────┼─────────┼────■─────────■──┤ X ├┤ X ├───────────────
«          │         │  ┌───┐  │  ┌─┴─┐          └───┘└───┘               
« anc_1: ──┼─────────■──┤ X ├──■──┤ X ├───────────────────────────────────
«          │  ┌───┐┌─┴─┐└───┘     └───┘                                   
« anc_2: ──■──┤ X ├┤ X ├──────────────────────────────────────────────────
«             └───┘└───┘                                                  
« anc_3: ─────────────────────────────────────────────────────────────────
«                                                                         
« anc_4: ──────────────────────────────────

In [41]:
from qiskit.quantum_info import Statevector

def test_addConstant(a, qubit_count, test_inputs):
    """
    For each x in test_inputs, initialise the data register to |x>,
    apply addConstant(a, qubit_count), and verify the data register
    holds |(x + a) mod 2^qubit_count>.
    Ancilla qubits are initialised to |0> and must return to |0>.
    """
    circ = addConstant(a, qubit_count)
    n_data = qubit_count
    n_anc  = circ.num_qubits - n_data
    results = []

    for x in test_inputs:
        # Build an initialisation circuit for data+ancilla
        init = QuantumCircuit(circ.num_qubits)
        for i in range(n_data):
            if (x >> i) & 1:
                init.x(i)          # data qubits are the first n_data

        full = init.compose(circ)
        sv   = Statevector(full)

        # Find the computational basis state with amplitude 1
        probs = sv.probabilities_dict()
        measured = max(probs, key=probs.get)   # bitstring, MSB first

        # Split into ancilla (high bits) and data (low bits)
        anc_bits  = measured[:n_anc]
        data_bits = measured[n_anc:]           # LSB of data is rightmost char

        data_val = int(data_bits, 2)
        expected = (x + a) % (2 ** n_data)
        ok = (data_val == expected) and (anc_bits == '0' * n_anc)
        results.append((x, data_val, expected, ok))
        print(f"x={x:3d}  got={data_val:3d}  expected={expected:3d}  ancilla_clean={anc_bits == '0'*n_anc}  {'OK' if ok else 'FAIL'}")

    all_ok = all(r[3] for r in results)
    print(f"\nAll tests passed: {all_ok}")
    return all_ok

# Test addConstant(150, 8) on several input values
test_addConstant(150, 8, test_inputs=[0, 1, 2, 5, 10, 50, 100, 105])


x=  0  got=150  expected=150  ancilla_clean=True  OK
x=  1  got=151  expected=151  ancilla_clean=True  OK
x=  2  got=152  expected=152  ancilla_clean=True  OK
x=  5  got=155  expected=155  ancilla_clean=True  OK
x= 10  got=160  expected=160  ancilla_clean=True  OK
x= 50  got=200  expected=200  ancilla_clean=True  OK
x=100  got=250  expected=250  ancilla_clean=True  OK
x=105  got=255  expected=255  ancilla_clean=True  OK

All tests passed: True


True